In [1]:
import pandas as pd
import pickle as pkl

In [2]:
all_rows_post = pd.read_pickle('all_rows_post2.pkl')

In [3]:
market_size = pd.read_excel('Extrap_byCZ_HARDI_Adjusted_27JAN2022 - Isaac Copy.xlsx' , sheet_name = 'Market Size', usecols='A:D')
sales_mix   = pd.read_pickle('sales_mix.pkl')

In [4]:
def get_market_pct_share(self, year, tech):
    numer = market_size.loc[(self['Year']== year)  & (self['Technology']== tech),['Quantity']].sum()
    denom = market_size.loc[(self['Year']== year),['Quantity']].sum()
    return numer/denom

In [5]:
def get_non_weighted_quantity(self, sector, year, tech, htg, clg):
    total = all_rows_post.loc[(self['Sector']== sector) & (self['yearSale']== year) & (self['Technology']== tech) & (self['HeatingEfficiency']== htg) & (self['CoolingEfficiency']== clg),['TOTAL']].sum()
    return round(total,1)

In [6]:
def get_pct_share(self, year, tech):
    numer = self['Quantity']
    denom = sales_mix.loc[(self['yearSale']== year) & (self['Technology']== tech),['Quantity']].sum()
    return numer/denom

In [7]:
def get_market_total(self, year, tech):
    total = market_size.loc[(self['yearSale'] == year) & (self['Technology']== tech),['Quantity']].sum()
    return total

In [8]:
def get_non_weighted_non_extrapped(self, sector, year, tech, htg, clg):
    total = all_rows_post.loc[(self['Sector']== sector) & (self['yearSale']== year) & (self['Technology']== tech) & (self['HeatingEfficiency']== htg) & (self['CoolingEfficiency']== clg),['Reported Quantity']].sum()
    return round(total,1)

In [9]:
def get_non_weighted_extrapped(self, sector, year, tech, htg, clg):
    total = all_rows_post.loc[(self['Sector']== sector) & (self['yearSale']== year) & (self['Technology']== tech) & (self['HeatingEfficiency']== htg) & (self['CoolingEfficiency']== clg),['extrapped_qty']].sum()
    return round(total,1)

In [10]:
all_rows_post = all_rows_post.replace('NA' , 'UNDEFINED')

In [11]:
market_size['TechMix'] = market_size.apply(get_market_pct_share, args=(market_size['Year'], market_size['Technology']), axis=1)

In [12]:
sales_mix['Quantity'] = round(sales_mix.apply(get_non_weighted_quantity, args=(all_rows_post['Sector'], all_rows_post['yearSale'], all_rows_post['Technology'], all_rows_post['HeatingEfficiency'] , all_rows_post['CoolingEfficiency']), axis=1),0)
sales_mix['%'] = sales_mix.apply(get_pct_share, args=(sales_mix['yearSale'], sales_mix['Technology']), axis=1)
sales_mix['Market Total'] = sales_mix.apply(get_market_total, args=(market_size['Year'], market_size['Technology']), axis=1)
sales_mix['Revised_Qty'] = round(sales_mix['Market Total'] * sales_mix['%'],0)
sales_mix['Non_Extrapped_no_weight'] = round(sales_mix.apply(get_non_weighted_non_extrapped, args=(all_rows_post['Sector'], all_rows_post['yearSale'], all_rows_post['Technology'], all_rows_post['HeatingEfficiency'] , all_rows_post['CoolingEfficiency']), axis=1),0)
sales_mix['Extrapped_no_weight'] = round(sales_mix.apply(get_non_weighted_extrapped, args=(all_rows_post['Sector'], all_rows_post['yearSale'], all_rows_post['Technology'], all_rows_post['HeatingEfficiency'] , all_rows_post['CoolingEfficiency']), axis=1),0)

In [13]:
sales_mix['Non_Extrapped_weighted'] = sales_mix['Revised_Qty'] * sales_mix['Non_Extrapped_no_weight'] / sales_mix['Quantity']
sales_mix['Extrapped_weighted']     = sales_mix['Revised_Qty'] * sales_mix['Extrapped_no_weight'] / sales_mix['Quantity']

In [14]:
key_techs = ['Heat Pump','Heat Pump - Ductless','Central Air Conditioning - Condenser','Gas Furnace']
df2_col_names = ['Year','CZ','Sector','Technology','HeatingEfficiency','CoolingEfficiency','Reported Quantity','Extrapolated Quantity','Total']
df1_col_names = ['Year','Sector','Technology','HeatingEfficiency','CoolingEfficiency','Total','Reported Quantity','Extrapolated Quantity','CZ']

In [15]:
sales_mix['CZ'] = ""
df1 = sales_mix.drop(['Quantity','%','Market Total','Non_Extrapped_no_weight','Extrapped_no_weight'],axis=1)
df1.columns = df1_col_names

In [16]:
df3 = all_rows_post[~all_rows_post['Technology'].isin(key_techs)]
df4 = all_rows_post[(all_rows_post['Sector'] == 'Non-residential')&(all_rows_post['Technology'].isin(key_techs))]
df2 = pd.concat([df3,df4] , axis=0)

In [17]:
df2=df2.groupby(['yearSale','CZ','Sector','Technology','HeatingEfficiency','CoolingEfficiency'])['Reported Quantity','extrapped_qty','TOTAL'].sum().reset_index()

C:\Users\ISAACS~1\AppData\Local\Temp/ipykernel_30740/4173003949.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2=df2.groupby(['yearSale','CZ','Sector','Technology','HeatingEfficiency','CoolingEfficiency'])['Reported Quantity','extrapped_qty','TOTAL'].sum().reset_index()


In [18]:
df2.columns=df2_col_names

In [19]:
all_rows_weighted = pd.concat([df1,df2] , axis=0)

In [20]:
weighted_quantites = all_rows_weighted.groupby(['Year','Technology'])['Total'].sum().reset_index()
weighted_quantites2 = all_rows_weighted[all_rows_weighted['Sector'] == 'Residential']
weighted_quantites2 = weighted_quantites2.groupby(['Year','Technology'])['Total'].sum().reset_index()

In [21]:
weighted_quantites.to_csv('All Technologies (weighted).csv')
weighted_quantites2.to_csv('All Res Technologies (weighted).csv')

In [22]:
res  = all_rows_weighted[all_rows_weighted['Sector'] == 'Residential']
res1 = res[(res['Technology'] == 'Heat Pump - Ductless') | (res['Technology'] == 'Heat Pump')|(res['Technology'] == 'Gas Furnace')]
res2 = res[(res['Technology'] == 'Heat Pump') | (res['Technology'] == 'Central Air Conditioning - Condenser')]
res1=res1[res1['HeatingEfficiency'] != 'UNDEFINED']
res_heating = res1.groupby(['Year','Technology','HeatingEfficiency'])['Total'].sum().reset_index()
res_cooling = res2.groupby(['Year','Technology','CoolingEfficiency'])['Total'].sum().reset_index()


years = [2016, 2017, 2018, 2019, 2020]
res_heat_eff_mix = pd.DataFrame()
res_cool_eff_mix = pd.DataFrame()


for y in years:
    df3 = res_heating[res_heating['Year'] == y]
    df4 = res_cooling[res_cooling['Year'] == y]
   
    ductless_total  = df3[df3['Technology'] == 'Heat Pump - Ductless']['Total'].sum()
    hp_total        = df4[df4['Technology'] == 'Heat Pump']['Total'].sum()
    furnace_total   = df3[df3['Technology'] == 'Gas Furnace']['Total'].sum()
    cac_total       = df4[df4['Technology'] == 'Central Air Conditioning - Condenser']['Total'].sum()

    df3.loc[df3['Technology'] == 'Heat Pump', 'pct_share']            = df3['Total'] / hp_total
    df3.loc[df3['Technology'] == 'Heat Pump - Ductless', 'pct_share'] = df3['Total'] / ductless_total
    df3.loc[df3['Technology'] == 'Gas Furnace', 'pct_share']          = df3['Total'] / furnace_total
    
    df4.loc[df4['Technology'] == 'Heat Pump', 'pct_share'] = df4['Total'] / hp_total
    df4.loc[df4['Technology'] == 'Central Air Conditioning - Condenser', 'pct_share'] = df4['Total'] / cac_total
    
    res_heat_eff_mix = pd.concat([res_heat_eff_mix, df3], axis=0)
    res_cool_eff_mix = pd.concat([res_cool_eff_mix, df4], axis=0)
    
res_heat_eff_mix.to_csv("Residential Heating Eff Mix (Weighted).csv")
res_cool_eff_mix.to_csv("Residential Cooling Eff Mix (Weighted).csv")

C:\Users\IsaacSchultz\anaconda3\lib\site-packages\pandas\core\indexing.py:1681: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = empty_value
C:\Users\IsaacSchultz\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [23]:
all_rows_weighted

,Year,Sector,Technology,HeatingEfficiency,CoolingEfficiency,Total,Reported Quantity,Extrapolated Quantity,CZ
8,2016,Residential,Central Air Conditioning - Condenser,UNDEFINED,SEER 10-11.99,849.0,849.000000,0.000000,
9,2016,Residential,Central Air Conditioning - Condenser,UNDEFINED,SEER 12-13.99,45468.0,37631.340203,7836.659797,
10,2016,Residential,Central Air Conditioning - Condenser,UNDEFINED,SEER 14-15.99,59487.0,38008.430224,21478.569776,
11,2016,Residential,Central Air Conditioning - Condenser,UNDEFINED,SEER 16 or above,8345.0,5351.048246,2993.951754,
13,2016,Residential,Gas Furnace,80% up to 89.99% AFUE,UNDEFINED,50328.0,40254.565441,10073.434559,
...,...,...,...,...,...,...,...,...,...
104,2020,Non-residential,Unitary Large Equipment,UNDEFINED,UNDEFINED,2115.0,305.000000,1810.000000,OR/WA
105,2020,Residential,Electric Furnace,UNDEFINED,UNDEFINED,1478.0,1478.000000,0.000000,OR/WA
106,2020,Residential,Gas Packaged Unit,UNDEFINED,UNDEFINED,2718.0,427.000000,2291.000000,OR/WA
107,2020,Residential,"Heat Pump - Air Source, Single Packaged",UNDEFINED,UNDEFINED,715.0,146.000000,569.000000,OR/WA


In [24]:
weighted_quantites['Total'].sum()

2496377.0